# Lab 02: Basic NLP Preprocessing Techniques

**Course:** ITAI 2373 - Natural Language Processing  
**Module:** 02 - Text Preprocessing  
**Duration:** 2-3 hours  
**Student Name:** _Jiri Musil_  
**Date:** 06/08/2025

---

## 🎯 Learning Objectives

By completing this lab, you will:
1. Understand the critical role of preprocessing in NLP pipelines
2. Master fundamental text preprocessing techniques
3. Compare different libraries and their approaches
4. Analyze the effects of preprocessing on text data
5. Build a complete preprocessing pipeline
6. Load and work with different types of text datasets

## 📖 Introduction to NLP Preprocessing

Natural Language Processing (NLP) preprocessing refers to the initial steps taken to clean and transform raw text data into a format that's more suitable for analysis by machine learning algorithms.

### Why is preprocessing crucial?

1. **Standardization:** Ensures consistent text format across your dataset
2. **Noise Reduction:** Removes irrelevant information that could confuse algorithms
3. **Complexity Reduction:** Simplifies text to focus on meaningful patterns
4. **Performance Enhancement:** Improves the efficiency and accuracy of downstream tasks

### Real-world Impact
Consider searching for "running shoes" vs "Running Shoes!" - without preprocessing, these might be treated as completely different queries. Preprocessing ensures they're recognized as equivalent.

### 🤔 Conceptual Question 1
**Before we start coding, think about your daily interactions with text processing systems (search engines, chatbots, translation apps). What challenges do you think these systems face when processing human language? List at least 3 specific challenges and explain why each is problematic.**

*Double-click this cell to write your answer:*

**Challenge 1:** Sometimes, I use abbreviations, emojis, varied punctuation, and other elements that introduce noise to the system, as the tokenizers and parsers expect clean, well-formatted text. This can break token boundaries, confuse engines, and degrade performance over time.

**Challenge 2:** Context and shared understanding are crucial since we often depend on information beyond individual sentences, such as previous discussions, cultural insights, customs, and the speaker's intention. Systems face challenges in tracking or following dialogue and world knowledge, leading to unhelpful answers.  

**Challenge 3:** Idioms and slang can vary significantly, making it particularly challenging for foreigners learning English to understand them hence poses significant issue for NLP systems. Expressions like “kick the bucket,” “spill the tea,” or localized slang often do not translate literally. Literal processing leads to incorrect output that lacks meaning. NLP systems need significant exposure to informal language that evolves over time. 

---

## 🛠️ Part 1: Environment Setup

We'll be working with two major NLP libraries:
- **NLTK (Natural Language Toolkit):** Comprehensive NLP library with extensive resources
- **spaCy:** Industrial-strength NLP with pre-trained models

**⚠️ Note:** Installation might take 2-3 minutes to complete.

In [1]:
# Step 1: Install Required Libraries
print("🔧 Installing NLP libraries...")

!pip install -q nltk spacy
!python -m spacy download en_core_web_sm

print("✅ Installation complete!")

🔧 Installing NLP libraries...
zsh:1: command not found: pip
zsh:1: command not found: python
✅ Installation complete!


### 🤔 Conceptual Question 2
**Why do you think we need to install a separate language model (en_core_web_sm) for spaCy? What components might this model contain that help with text processing? Think about what information a computer needs to understand English text.**

The en_core_web_sm model provides English-specific vocabulary, tokenization rules, and morphological features that are not included in spaCy’s core library alone. It also provides pretrained statistical weights for part-of-speech tagging, dependency parsing, and named entity recognition. Without this model, spaCy is missing the essential language knowledge required to accurately split, tag, and comprehend English text.



In [2]:
# Step 2: Import Libraries and Download NLTK Data
import nltk
import spacy
import string
import re
from collections import Counter

# Download essential NLTK data
print("📦 Downloading NLTK data packages...")
nltk.download('punkt')      # For tokenization
nltk.download('punkt_tab')  # the missing piece
nltk.download('stopwords')  # For stop word removal
nltk.download('wordnet')    # For lemmatization
nltk.download('averaged_perceptron_tagger')  # For POS tagging

print("\n✅ All imports and downloads completed!")

/Users/jirimusil/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
[nltk_data] Downloading package punkt to /Users/jirimusil/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/jirimusil/nltk_data...


📦 Downloading NLTK data packages...

✅ All imports and downloads completed!


[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jirimusil/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jirimusil/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/jirimusil/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


## 📂 Part 2: Sample Text Data

We'll work with different types of text to understand how preprocessing affects various text styles:
- Simple text
- Academic text (with citations, URLs)
- Social media text (with emojis, hashtags)
- News text (formal writing)
- Product reviews (informal, ratings)

In [3]:
# Step 3: Load Sample Texts
simple_text = "Natural Language Processing is a fascinating field of AI. It's amazing!"

academic_text = """
Dr. Smith's research on machine-learning algorithms is groundbreaking!
She published 3 papers in 2023, focusing on deep neural networks (DNNs).
The results were amazing - accuracy improved by 15.7%!
"This is revolutionary," said Prof. Johnson.
Visit https://example.com for more info. #NLP #AI @university
"""

social_text = "OMG! Just tried the new coffee shop ☕️ SO GOOD!!! Highly recommend 👍 #coffee #yum 😍"

news_text = """
The stock market experienced significant volatility today, with tech stocks leading the decline.
Apple Inc. (AAPL) dropped 3.2%, while Microsoft Corp. fell 2.8%.
"We're seeing a rotation out of growth stocks," said analyst Jane Doe from XYZ Capital.
"""

review_text = """
This laptop is absolutely fantastic! I've been using it for 6 months and it's still super fast.
The battery life is incredible - lasts 8-10 hours easily.
Only complaint: the keyboard could be better. Overall rating: 4.5/5 stars.
"""

# Store all texts
sample_texts = {
    "Simple": simple_text,
    "Academic": academic_text.strip(),
    "Social Media": social_text,
    "News": news_text.strip(),
    "Product Review": review_text.strip()
}

print("📄 Sample texts loaded successfully!")
for name, text in sample_texts.items():
    preview = text[:80] + "..." if len(text) > 80 else text
    print(f"\n🏷️ {name}: {preview}")

📄 Sample texts loaded successfully!

🏷️ Simple: Natural Language Processing is a fascinating field of AI. It's amazing!

🏷️ Academic: Dr. Smith's research on machine-learning algorithms is groundbreaking!
She publi...

🏷️ Social Media: OMG! Just tried the new coffee shop ☕️ SO GOOD!!! Highly recommend 👍 #coffee #yu...

🏷️ News: The stock market experienced significant volatility today, with tech stocks lead...

🏷️ Product Review: This laptop is absolutely fantastic! I've been using it for 6 months and it's st...


### 🤔 Conceptual Question 3
**Looking at the different text types we've loaded, what preprocessing challenges do you anticipate for each type? For each text type below, identify at least 2 specific preprocessing challenges and explain why they might be problematic for NLP analysis.**

*Double-click this cell to write your answer:*

**Simple text challenges:**
1.Contractions and apostrophes, looking at “It’s” should be divided into “It” and ‘s’ or possibly expanded to “it is”  for accurate tokenization and part-of-speech tagging.
2.Sentence boundary detection, in the text we have: The period following “AI." as opposed to "!” in “field of AI. It’s amazing!” which necessitates proper recognition of two distinct sentences, instead of incorrectly merging or excessively dividing them.

**Academic text challenges:**
1. Abbreviations and Acronyms, specifically terms such as "DNNs” and titles like "Dr." and “Prof." should be acknowledged as single tokens to avoid errors which  splits or misinterpret as sentence boundaries.
2.Here is another aspect in the form of Inline Citations, URLs & Hashtags,  elements such as “https://example.com”, “#NLP”, and “@university” contain non-standard tokens that may confuse tokenizers and models.

**Social media text challenges:**
1.Emojis & Emoticons, such as “☕️”, “👍”, and “😍", express sentiment but are outside standard vocabularies. They need to be thoughtfully mapped to sentiment tokens or removed.
2.Informal Language & Punctuation Excess, things like abbreviations (e.g., "OMG”), excessive capitalization (“SO GOOD!!!”) and multiple exclamation marks need to be standardized to prevent skewed sentiment and inaccurate token statistics.

**News text challenges:**
1.Numeric data and symbols, such as percentages (“3.2%”) and ticker symbols (“AAPL"), must be processed as structured entities rather than generic tokens for tasks like information extraction.
2.Quoted Speech and Attribution. Proper segmentation of nested quotations (e.g., “We’re seeing…,” said analyst Jane Doe) is essential to clearly differentiate reported speech from the narrative text.

**Product review challenges:**
1.Rating expressions such as “4.5/5 stars” convey quantitative feedback, which should be extracted as a numerical feature instead of being considered regular text.
2.Colloquial versus Formal Phrases poses significant challenge. Expressions such as “absolutely fantastic” compared to “Only complaint:” necessitate uniform sentiment normalization and effective management of list-like formats.

---

In [4]:
from nltk.tokenize import word_tokenize, sent_tokenize

# now this will work
tokens = word_tokenize("It's amazing!")
sentences = sent_tokenize("Natural Language Processing is fun. It's amazing!")

## 🔤 Part 3: Tokenization

### What is Tokenization?
Tokenization is the process of breaking down text into smaller, meaningful units called **tokens**. These tokens are typically words, but can also be sentences, characters, or subwords.

### Why is it Important?
- Most NLP algorithms work with individual tokens, not entire texts
- It's the foundation for all subsequent preprocessing steps
- Different tokenization strategies can significantly impact results

### Common Challenges:
- **Contractions:** "don't" → "do" + "n't" or "don't"?
- **Punctuation:** Keep with words or separate?
- **Special characters:** How to handle @, #, URLs?

In [5]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("ChatGPT helps with Python code. It's quite useful!")
tokens = [token.text for token in doc]

print(tokens)

['ChatGPT', 'helps', 'with', 'Python', 'code', '.', 'It', "'s", 'quite', 'useful', '!']


In [6]:
# Step 4: Tokenization with NLTK
from nltk.tokenize import word_tokenize, sent_tokenize

# Test on simple text
print("🔍 NLTK Tokenization Results")
print("=" * 40)
print(f"Original: {simple_text}")

# Word tokenization
nltk_tokens = word_tokenize(simple_text)
print(f"\nWord tokens: {nltk_tokens}")
print(f"Number of tokens: {len(nltk_tokens)}")

# Sentence tokenization
sentences = sent_tokenize(simple_text)
print(f"\nSentences: {sentences}")
print(f"Number of sentences: {len(sentences)}")


🔍 NLTK Tokenization Results
Original: Natural Language Processing is a fascinating field of AI. It's amazing!

Word tokens: ['Natural', 'Language', 'Processing', 'is', 'a', 'fascinating', 'field', 'of', 'AI', '.', 'It', "'s", 'amazing', '!']
Number of tokens: 14

Sentences: ['Natural Language Processing is a fascinating field of AI.', "It's amazing!"]
Number of sentences: 2


### 🤔 Conceptual Question 4
**Examine the NLTK tokenization results above. How did NLTK handle the contraction "It's"? What happened to the punctuation marks? Do you think this approach is appropriate for all NLP tasks? Explain your reasoning.**

*Double-click this cell to write your answer:*

**How "It's" was handled:** I noticed NLTK split “It’s” into two tokens: “It” and “’s.”

**Punctuation treatment:** I saw that apostrophes and periods were pulled out as separate tokens rather than kept attached.

**Appropriateness for different tasks:**I think that works fine for parsing and tagging, but for tasks like sentiment analysis or text generation I might prefer to keep contractions intact or expand them for clarity.

---

In [7]:
# Step 5: Tokenization with spaCy
nlp = spacy.load('en_core_web_sm')

print("🔍 spaCy Tokenization Results")
print("=" * 40)
print(f"Original: {simple_text}")

# Process with spaCy
doc = nlp(simple_text)

# Extract tokens
spacy_tokens = [token.text for token in doc]
print(f"\nWord tokens: {spacy_tokens}")
print(f"Number of tokens: {len(spacy_tokens)}")

# Show detailed token information
print(f"\n🔬 Detailed Token Analysis:")
print(f"{'Token':<12} {'POS':<8} {'Lemma':<12} {'Is Alpha':<8} {'Is Stop':<8}")
print("-" * 50)
for token in doc:
    print(f"{token.text:<12} {token.pos_:<8} {token.lemma_:<12} {token.is_alpha:<8} {token.is_stop:<8}")

🔍 spaCy Tokenization Results
Original: Natural Language Processing is a fascinating field of AI. It's amazing!

Word tokens: ['Natural', 'Language', 'Processing', 'is', 'a', 'fascinating', 'field', 'of', 'AI', '.', 'It', "'s", 'amazing', '!']
Number of tokens: 14

🔬 Detailed Token Analysis:
Token        POS      Lemma        Is Alpha Is Stop 
--------------------------------------------------
Natural      PROPN    Natural      1        0       
Language     PROPN    Language     1        0       
Processing   NOUN     processing   1        0       
is           AUX      be           1        1       
a            DET      a            1        1       
fascinating  ADJ      fascinating  1        0       
field        NOUN     field        1        0       
of           ADP      of           1        1       
AI           PROPN    AI           1        0       
.            PUNCT    .            0        0       
It           PRON     it           1        1       
's           AUX     

### 🤔 Conceptual Question 5
**Compare the NLTK and spaCy tokenization results. What differences do you notice? Which approach do you think would be better for different NLP tasks? Consider specific examples like sentiment analysis vs. information extraction.**

*Double-click this cell to write your answer:*

**Key differences observed:** I noticed that both NLTK and spaCy split “It’s” into “It” and “’s,” but spaCy goes further by tagging each token with part-of-speech labels, lemmas, and stop-word flags. In contrast, NLTK simply provides raw tokens without any linguistic details.

**Better for sentiment analysis:** For sentiment analysis, I would prefer NLTK’s simpler tokens or a specialized lexicon approach, as I believe that I would often need clean word lists with minimal details. I am a big fan of KISS approach – keep it as simple as possible. 

**Better for information extraction:** I am not an expert yet, but I believe that in terms of extracting entities or relationships, spaCy is superior because it includes POS tags, dependency parses, and other elements that provide the structure I could need to reliably extract facts.

**Overall assessment:** I would turn to spaCy for its comprehensive linguistic features and well-structured output. However, if I were in need of quick token lists or in resource-limited environments, I would use NLTK's lightweight tokenizer.

---

In [8]:
# Step 6: Test Tokenization on Complex Text
print("🧪 Testing on Social Media Text")
print("=" * 40)
print(f"Original: {social_text}")

# NLTK approach
social_nltk_tokens = word_tokenize(social_text)
print(f"\nNLTK tokens: {social_nltk_tokens}")

# spaCy approach
social_doc = nlp(social_text)
social_spacy_tokens = [token.text for token in social_doc]
print(f"spaCy tokens: {social_spacy_tokens}")

print(f"\n📊 Comparison:")
print(f"NLTK token count: {len(social_nltk_tokens)}")
print(f"spaCy token count: {len(social_spacy_tokens)}")

🧪 Testing on Social Media Text
Original: OMG! Just tried the new coffee shop ☕️ SO GOOD!!! Highly recommend 👍 #coffee #yum 😍

NLTK tokens: ['OMG', '!', 'Just', 'tried', 'the', 'new', 'coffee', 'shop', '☕️', 'SO', 'GOOD', '!', '!', '!', 'Highly', 'recommend', '👍', '#', 'coffee', '#', 'yum', '😍']
spaCy tokens: ['OMG', '!', 'Just', 'tried', 'the', 'new', 'coffee', 'shop', '☕', '️', 'SO', 'GOOD', '!', '!', '!', 'Highly', 'recommend', '👍', '#', 'coffee', '#', 'yum', '😍']

📊 Comparison:
NLTK token count: 22
spaCy token count: 23


### 🤔 Conceptual Question 6
**Looking at how the libraries handled social media text (emojis, hashtags), which library seems more robust for handling "messy" real-world text? What specific advantages do you notice? How might this impact a real-world application like social media sentiment analysis?**

*Double-click this cell to write your answer:*

**More robust library:** I believe spaCy manages messy real-world text more effectively than NLTK.

**Specific advantages:** It keeps emojis, intelligently separates combining marks, and treats hashtags as single tokens rather than isolating the “#” symbol.

**Impact on sentiment analysis:** This means that I can capture emoji-driven sentiment and hashtag cues more reliably, resulting in cleaner inputs and more accurate mood scoring.

---

## 🛑 Part 4: Stop Words Removal

### What are Stop Words?
Stop words are common words that appear frequently in a language but typically don't carry much meaningful information about the content. Examples include "the", "is", "at", "which", "on", etc.

### Why Remove Stop Words?
1. **Reduce noise** in the data
2. **Improve efficiency** by reducing vocabulary size
3. **Focus on content words** that carry semantic meaning

### When NOT to Remove Stop Words?
- **Sentiment analysis:** "not good" vs "good" - the "not" is crucial!
- **Question answering:** "What is the capital?" - "what" and "is" provide context

In [9]:
# Step 7: Explore Stop Words Lists
from nltk.corpus import stopwords

# Get NLTK English stop words
nltk_stopwords = set(stopwords.words('english'))
print(f"📊 NLTK has {len(nltk_stopwords)} English stop words")
print(f"First 20: {sorted(list(nltk_stopwords))[:20]}")

# Get spaCy stop words
spacy_stopwords = nlp.Defaults.stop_words
print(f"\n📊 spaCy has {len(spacy_stopwords)} English stop words")
print(f"First 20: {sorted(list(spacy_stopwords))[:20]}")

# Compare the lists
common_stopwords = nltk_stopwords.intersection(spacy_stopwords)
nltk_only = nltk_stopwords - spacy_stopwords
spacy_only = spacy_stopwords - nltk_stopwords

print(f"\n🔍 Comparison:")
print(f"Common stop words: {len(common_stopwords)}")
print(f"Only in NLTK: {len(nltk_only)} - Examples: {sorted(list(nltk_only))[:5]}")
print(f"Only in spaCy: {len(spacy_only)} - Examples: {sorted(list(spacy_only))[:5]}")

📊 NLTK has 198 English stop words
First 20: ['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been']

📊 spaCy has 326 English stop words
First 20: ["'d", "'ll", "'m", "'re", "'s", "'ve", 'a', 'about', 'above', 'across', 'after', 'afterwards', 'again', 'against', 'all', 'almost', 'alone', 'along', 'already', 'also']

🔍 Comparison:
Common stop words: 123
Only in NLTK: 75 - Examples: ['ain', 'aren', "aren't", 'couldn', "couldn't"]
Only in spaCy: 203 - Examples: ["'d", "'ll", "'m", "'re", "'s"]


### 🤔 Conceptual Question 7
**Why do you think NLTK and spaCy have different stop word lists? Look at the examples of words that are only in one list - do you agree with these choices? Can you think of scenarios where these differences might significantly impact your NLP results?**

*Double-click this cell to write your answer:*

**Reasons for differences:** I believe every library curates its list to align with its design objectives. NLTK focuses on a minimal core set of grammatical words, whereas spaCy incorporates more contractions and other forms, enabling its models to remove common affixes.

**Agreement with choices:** I agree that spaCy’s inclusion of “’s” and “’ve” is logical for pipelines aiming to normalize or eliminate contractions, while NLTK’s list appears more conservative by concentrating on complete words.

**Scenarios where differences matter:** When performing topic modeling, spaCy’s more extensive list is useful for eliminating noise from contractions. However, for analyzing dialogue authenticity, I would choose to retain those clitic(had to google right term - affix, a phonologically dependent word) forms and would use NLTK’s shorter list to avoid unintentionally removing significant nuances.

---

In [10]:
# Step 8: Remove Stop Words with NLTK
# Test on simple text
original_tokens = nltk_tokens  # From earlier tokenization
filtered_tokens = [word for word in original_tokens if word.lower() not in nltk_stopwords]

print("🧪 NLTK Stop Word Removal")
print("=" * 40)
print(f"Original: {simple_text}")
print(f"\nOriginal tokens ({len(original_tokens)}): {original_tokens}")
print(f"After removing stop words ({len(filtered_tokens)}): {filtered_tokens}")

# Show which words were removed
removed_words = [word for word in original_tokens if word.lower() in nltk_stopwords]
print(f"\nRemoved words: {removed_words}")

# Calculate reduction percentage
reduction = (len(original_tokens) - len(filtered_tokens)) / len(original_tokens) * 100
print(f"Vocabulary reduction: {reduction:.1f}%")

🧪 NLTK Stop Word Removal
Original: Natural Language Processing is a fascinating field of AI. It's amazing!

Original tokens (14): ['Natural', 'Language', 'Processing', 'is', 'a', 'fascinating', 'field', 'of', 'AI', '.', 'It', "'s", 'amazing', '!']
After removing stop words (10): ['Natural', 'Language', 'Processing', 'fascinating', 'field', 'AI', '.', "'s", 'amazing', '!']

Removed words: ['is', 'a', 'of', 'It']
Vocabulary reduction: 28.6%


In [11]:
# Step 9: Remove Stop Words with spaCy
doc = nlp(simple_text)
spacy_filtered = [token.text for token in doc if not token.is_stop and not token.is_punct]

print("🧪 spaCy Stop Word Removal")
print("=" * 40)
print(f"Original: {simple_text}")
print(f"\nOriginal tokens ({len(spacy_tokens)}): {spacy_tokens}")
print(f"After removing stop words & punctuation ({len(spacy_filtered)}): {spacy_filtered}")

# Show which words were removed
spacy_removed = [token.text for token in doc if token.is_stop or token.is_punct]
print(f"\nRemoved words: {spacy_removed}")

# Calculate reduction percentage
spacy_reduction = (len(spacy_tokens) - len(spacy_filtered)) / len(spacy_tokens) * 100
print(f"Vocabulary reduction: {spacy_reduction:.1f}%")

🧪 spaCy Stop Word Removal
Original: Natural Language Processing is a fascinating field of AI. It's amazing!

Original tokens (14): ['Natural', 'Language', 'Processing', 'is', 'a', 'fascinating', 'field', 'of', 'AI', '.', 'It', "'s", 'amazing', '!']
After removing stop words & punctuation (7): ['Natural', 'Language', 'Processing', 'fascinating', 'field', 'AI', 'amazing']

Removed words: ['is', 'a', 'of', '.', 'It', "'s", '!']
Vocabulary reduction: 50.0%


### 🤔 Conceptual Question 8
**Compare the NLTK and spaCy stop word removal results. Which approach removed more words? Do you think removing punctuation (as spaCy did) is always a good idea? Give a specific example where keeping punctuation might be important for NLP analysis.**

*Double-click this cell to write your answer:*

**Which removed more:** SpaCy’s pipeline eliminated half the tokens, while NLTK reduced them by 28.6%, meaning SpaCy removed more words overall.

**Punctuation removal assessment:** I don’t believe that dropping punctuation is always wise. Those little marks often convey tone or structural clues that words alone can’t express.

**Example where punctuation matters:** In customer support transcripts, a question mark indicates a user request, while an exclamation mark can convey urgency or frustration. If we were to remove these, it would eliminate important signals.

---

## 🌱 Part 5: Lemmatization and Stemming

### What is Lemmatization?
Lemmatization reduces words to their base or dictionary form (called a **lemma**). It considers context and part of speech to ensure the result is a valid word.

### What is Stemming?
Stemming reduces words to their root form by removing suffixes. It's faster but less accurate than lemmatization.

### Key Differences:
| Aspect | Stemming | Lemmatization |
|--------|----------|---------------|
| Speed | Fast | Slower |
| Accuracy | Lower | Higher |
| Output | May be non-words | Always valid words |
| Context | Ignores context | Considers context |

### Examples:
- **"running"** → Stem: "run", Lemma: "run"
- **"better"** → Stem: "better", Lemma: "good"
- **"was"** → Stem: "wa", Lemma: "be"

In [12]:
# Step 10: Stemming with NLTK
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()

# Test words that demonstrate stemming challenges
test_words = ['running', 'runs', 'ran', 'better', 'good', 'best', 'flying', 'flies', 'was', 'were', 'cats', 'dogs']

print("🌿 Stemming Demonstration")
print("=" * 30)
print(f"{'Original':<12} {'Stemmed':<12}")
print("-" * 25)

for word in test_words:
    stemmed = stemmer.stem(word)
    print(f"{word:<12} {stemmed:<12}")

# Apply to our sample text
sample_tokens = [token for token in nltk_tokens if token.isalpha()]
stemmed_tokens = [stemmer.stem(token.lower()) for token in sample_tokens]

print(f"\n🧪 Applied to sample text:")
print(f"Original: {sample_tokens}")
print(f"Stemmed: {stemmed_tokens}")

🌿 Stemming Demonstration
Original     Stemmed     
-------------------------
running      run         
runs         run         
ran          ran         
better       better      
good         good        
best         best        
flying       fli         
flies        fli         
was          wa          
were         were        
cats         cat         
dogs         dog         

🧪 Applied to sample text:
Original: ['Natural', 'Language', 'Processing', 'is', 'a', 'fascinating', 'field', 'of', 'AI', 'It', 'amazing']
Stemmed: ['natur', 'languag', 'process', 'is', 'a', 'fascin', 'field', 'of', 'ai', 'it', 'amaz']


### 🤔 Conceptual Question 9
**Look at the stemming results above. Can you identify any cases where stemming produced questionable results? For example, how were "better" and "good" handled? Do you think this is problematic for NLP applications? Explain your reasoning.**

*Double-click this cell to write your answer:*

**Questionable results identified:** I observed that “amazing” was shortened to “amaz” and “fascinating” to “fascin,” which seems unnatural and strips away meaning. I also noted that “ran” remained as “ran” while its present tense form changed to “run,” resulting in inconsistent roots.

**Assessment of "better" and "good":** Both “better” and “good” remain unchanged, so the comparative and base forms aren’t standardized in the same manner as regular verbs. This inconsistency implies that related words won’t be categorized together.

**Impact on NLP applications:** In sentiment analysis or search, I’m concerned that stems like “amaz” may not align with sentiment or user inquiries. Such mismatched mappings could fragment the feature space and negatively impact model performance.

---

In [13]:
# Step 11: Lemmatization with spaCy
print("🌱 spaCy Lemmatization Demonstration")
print("=" * 40)

# Test on a complex sentence
complex_sentence = "The researchers were studying the effects of running and swimming on better performance."
doc = nlp(complex_sentence)

print(f"Original: {complex_sentence}")
print(f"\n{'Token':<15} {'Lemma':<15} {'POS':<10} {'Explanation':<20}")
print("-" * 65)

for token in doc:
    if token.is_alpha:
        explanation = "No change" if token.text.lower() == token.lemma_ else "Lemmatized"
        print(f"{token.text:<15} {token.lemma_:<15} {token.pos_:<10} {explanation:<20}")

# Extract lemmas
lemmas = [token.lemma_.lower() for token in doc if token.is_alpha and not token.is_stop]
print(f"\n🔤 Lemmatized tokens (no stop words): {lemmas}")

🌱 spaCy Lemmatization Demonstration
Original: The researchers were studying the effects of running and swimming on better performance.

Token           Lemma           POS        Explanation         
-----------------------------------------------------------------
The             the             DET        No change           
researchers     researcher      NOUN       Lemmatized          
were            be              AUX        Lemmatized          
studying        study           VERB       Lemmatized          
the             the             DET        No change           
effects         effect          NOUN       Lemmatized          
of              of              ADP        No change           
running         run             VERB       Lemmatized          
and             and             CCONJ      No change           
swimming        swim            VERB       Lemmatized          
on              on              ADP        No change           
better          well          

In [14]:
# Step 12: Compare Stemming vs Lemmatization
comparison_words = ['better', 'running', 'studies', 'was', 'children', 'feet']

print("⚖️ Stemming vs Lemmatization Comparison")
print("=" * 50)
print(f"{'Original':<12} {'Stemmed':<12} {'Lemmatized':<12}")
print("-" * 40)

for word in comparison_words:
    # Stemming
    stemmed = stemmer.stem(word)

    # Lemmatization with spaCy
    doc = nlp(word)
    lemmatized = doc[0].lemma_

    print(f"{word:<12} {stemmed:<12} {lemmatized:<12}")

⚖️ Stemming vs Lemmatization Comparison
Original     Stemmed      Lemmatized  
----------------------------------------
better       better       well        
running      run          run         
studies      studi        study       
was          wa           be          
children     children     child       
feet         feet         foot        


### 🤔 Conceptual Question 10
**Compare the stemming and lemmatization results. Which approach do you think is more suitable for:**
1. **A search engine** (where speed is crucial and you need to match variations of words)?
2. **A sentiment analysis system** (where accuracy and meaning preservation are important)?
3. **A real-time chatbot** (where both speed and accuracy matter)?

**Explain your reasoning for each choice.**

*Double-click this cell to write your answer:*

**1. Search engine:** I would lean on stemming because it is fast and still matches word variants, allowing me to find “run,” “running,” or “runs” all at once. The slight roughness in stems does not negatively impact basic keyword matching.

**2. Sentiment analysis:** I would  go with lemmatization because keeping words in their correct form is important for tone and nuance. Turning “better” into “well” and “studying” into “study” ensures my sentiment remains accurate.

**3. Real-time chatbot:** I would also use lemmatization, but with a lightweight pipeline to achieve fairly accurate language understanding without noticeable lag. This balance keeps responses both quick and sensible.

---

## 🧹 Part 6: Text Cleaning and Normalization

### What is Text Cleaning?
Text cleaning involves removing or standardizing elements that might interfere with analysis:
- **Case normalization** (converting to lowercase)
- **Punctuation removal**
- **Number handling** (remove, replace, or normalize)
- **Special character handling** (URLs, emails, mentions)
- **Whitespace normalization**

### Why is it Important?
- Ensures consistency across your dataset
- Reduces vocabulary size
- Improves model performance
- Handles edge cases in real-world data

In [15]:
# Step 13: Basic Text Cleaning
def basic_clean_text(text):
    """Apply basic text cleaning operations"""
    # Convert to lowercase
    text = text.lower()

    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Remove extra spaces again
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# Test basic cleaning
test_text = "   Hello WORLD!!! This has 123 numbers and   extra spaces.   "
cleaned = basic_clean_text(test_text)

print("🧹 Basic Text Cleaning")
print("=" * 30)
print(f"Original: '{test_text}'")
print(f"Cleaned: '{cleaned}'")
print(f"Length reduction: {(len(test_text) - len(cleaned))/len(test_text)*100:.1f}%")

🧹 Basic Text Cleaning
Original: '   Hello WORLD!!! This has 123 numbers and   extra spaces.   '
Cleaned: 'hello world this has numbers and extra spaces'
Length reduction: 26.2%


In [16]:
# Step 14: Advanced Cleaning for Social Media
def advanced_clean_text(text):
    """Apply advanced cleaning for social media and web text"""
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Remove email addresses
    text = re.sub(r'\S+@\S+', '', text)

    # Remove mentions (@username)
    text = re.sub(r'@\w+', '', text)

    # Convert hashtags (keep the word, remove #)
    text = re.sub(r'#(\w+)', r'\1', text)

    # Remove emojis (basic approach)
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)

    # Convert to lowercase and normalize whitespace
    text = text.lower()
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# Test on social media text
print("🚀 Advanced Cleaning on Social Media Text")
print("=" * 45)
print(f"Original: {social_text}")

cleaned_social = advanced_clean_text(social_text)
print(f"Cleaned: {cleaned_social}")
print(f"Length reduction: {(len(social_text) - len(cleaned_social))/len(social_text)*100:.1f}%")

🚀 Advanced Cleaning on Social Media Text
Original: OMG! Just tried the new coffee shop ☕️ SO GOOD!!! Highly recommend 👍 #coffee #yum 😍
Cleaned: omg! just tried the new coffee shop ☕️ so good!!! highly recommend coffee yum
Length reduction: 7.2%


### 🤔 Conceptual Question 11
**Look at the advanced cleaning results for the social media text. What information was lost during cleaning? Can you think of scenarios where removing emojis and hashtags might actually hurt your NLP application? What about scenarios where keeping them would be beneficial?**

*Double-click this cell to write your answer:*

**Information lost:** It eliminated emojis that conveyed sentiment and hashtags that indicated topics or trends. It also removed mentions and URLs that might have referenced users or provided external context.

**Scenarios where removal hurts:** If I’m conducting social media sentiment analysis, losing emojis like 😂 or 😡 can erase key mood signals. Excluding hashtags also blinds me to campaign or community tags that help group posts by theme.

**Scenarios where keeping helps:** In a named entity recognition task, I might choose to disregard noisy hashtags and concentrate on clearer text. Similarly, when training a general language model, eliminating emojis and tags can help decrease vocabulary excess and accelerate training. 

---

## 🔧 Part 7: Building a Complete Preprocessing Pipeline

Now let's combine everything into a comprehensive preprocessing pipeline that you can customize based on your needs.

### Pipeline Components:
1. **Text cleaning** (basic or advanced)
2. **Tokenization** (NLTK or spaCy)
3. **Stop word removal** (optional)
4. **Lemmatization/Stemming** (optional)
5. **Additional filtering** (length, etc.)

In [17]:
# Step 15: Complete Preprocessing Pipeline
def preprocess_text(text,
                   clean_level='basic',     # 'basic' or 'advanced'
                   remove_stopwords=True,
                   use_lemmatization=True,
                   use_stemming=False,
                   min_length=2):
    """
    Complete text preprocessing pipeline
    """
    # Step 1: Clean text
    if clean_level == 'basic':
        cleaned_text = basic_clean_text(text)
    else:
        cleaned_text = advanced_clean_text(text)

    # Step 2: Tokenize
    if use_lemmatization:
        # Use spaCy for lemmatization
        doc = nlp(cleaned_text)
        tokens = [token.lemma_.lower() for token in doc if token.is_alpha]
    else:
        # Use NLTK for basic tokenization
        tokens = word_tokenize(cleaned_text)
        tokens = [token for token in tokens if token.isalpha()]

    # Step 3: Remove stop words
    if remove_stopwords:
        if use_lemmatization:
            tokens = [token for token in tokens if token not in spacy_stopwords]
        else:
            tokens = [token.lower() for token in tokens if token.lower() not in nltk_stopwords]

    # Step 4: Apply stemming if requested
    if use_stemming and not use_lemmatization:
        tokens = [stemmer.stem(token.lower()) for token in tokens]

    # Step 5: Filter by length
    tokens = [token for token in tokens if len(token) >= min_length]

    return tokens

print("🔧 Preprocessing Pipeline Created!")
print("✅ Ready to test different configurations.")

🔧 Preprocessing Pipeline Created!
✅ Ready to test different configurations.


In [18]:
# Step 16: Test Different Pipeline Configurations
test_text = sample_texts["Product Review"]
print(f"🎯 Testing on: {test_text[:100]}...")
print("=" * 60)

# Configuration 1: Minimal processing
minimal = preprocess_text(test_text,
                         clean_level='basic',
                         remove_stopwords=False,
                         use_lemmatization=False,
                         use_stemming=False)
print(f"\n1. Minimal processing ({len(minimal)} tokens):")
print(f"   {minimal[:10]}...")

# Configuration 2: Standard processing
standard = preprocess_text(test_text,
                          clean_level='basic',
                          remove_stopwords=True,
                          use_lemmatization=True)
print(f"\n2. Standard processing ({len(standard)} tokens):")
print(f"   {standard[:10]}...")

# Configuration 3: Aggressive processing
aggressive = preprocess_text(test_text,
                            clean_level='advanced',
                            remove_stopwords=True,
                            use_lemmatization=False,
                            use_stemming=True,
                            min_length=3)
print(f"\n3. Aggressive processing ({len(aggressive)} tokens):")
print(f"   {aggressive[:10]}...")

# Show reduction percentages
original_count = len(word_tokenize(test_text))
print(f"\n📊 Token Reduction Summary:")
print(f"   Original: {original_count} tokens")
print(f"   Minimal: {len(minimal)} ({(original_count-len(minimal))/original_count*100:.1f}% reduction)")
print(f"   Standard: {len(standard)} ({(original_count-len(standard))/original_count*100:.1f}% reduction)")
print(f"   Aggressive: {len(aggressive)} ({(original_count-len(aggressive))/original_count*100:.1f}% reduction)")

🎯 Testing on: This laptop is absolutely fantastic! I've been using it for 6 months and it's still super fast.
The ...

1. Minimal processing (34 tokens):
   ['this', 'laptop', 'is', 'absolutely', 'fantastic', 'ive', 'been', 'using', 'it', 'for']...

2. Standard processing (18 tokens):
   ['laptop', 'absolutely', 'fantastic', 've', 'use', 'month', 'super', 'fast', 'battery', 'life']...

3. Aggressive processing (21 tokens):
   ['laptop', 'absolut', 'fantast', 'use', 'month', 'still', 'super', 'fast', 'batteri', 'life']...

📊 Token Reduction Summary:
   Original: 47 tokens
   Minimal: 34 (27.7% reduction)
   Standard: 18 (61.7% reduction)
   Aggressive: 21 (55.3% reduction)


### 🤔 Conceptual Question 12
**Compare the three pipeline configurations (Minimal, Standard, Aggressive). For each configuration, analyze:**
1. **What information was preserved?**
2. **What information was lost?**
3. **What type of NLP task would this configuration be best suited for?**

*Double-click this cell to write your answer:*

**Minimal Processing:**
- Preserved: All original words, including stop words like “is,” “it,” and “for,” as well as pronouns and exact phrasing (“I’ve” and “been.”).
- Lost: Very little, mostly just basic lowercasing, while punctuation and contractions remain largely intact.
- Best for: Tasks that require complete context or pronoun resolution, such as reference tracking or extractive summarization.

**Standard Processing:**
- Preserved:Key content words: “laptop,” “fantastic,” “battery,” “life”, basic lemmas such as “use” and “month.”
- Lost: Most stop words and contractions are separated ("I’ve” becomes “ve"), along with finer elements like pronouns and auxiliary verbs.
- Best for: Topic modeling or document classification where the emphasis is on thematic terms, disregarding filler words.

**Aggressive Processing:**
- Preserved: Root word stems (“laptop,” “fantast,” “batteri,” “life”), ensuring that different forms connect correctly.
- Lost: Subtle variations in word forms ( “absolutely” → “absolut” ), time indicators (“still”), and clear phrasing.
- Best for: Searches that focus on finding any version of a word rather than worrying about how easy it is to read.

---

In [19]:
# Step 17: Comprehensive Analysis Across Text Types
print("🔬 Comprehensive Preprocessing Analysis")
print("=" * 50)

# Test standard preprocessing on all text types
results = {}
for name, text in sample_texts.items():
    original_tokens = len(word_tokenize(text))
    processed_tokens = preprocess_text(text,
                                      clean_level='basic',
                                      remove_stopwords=True,
                                      use_lemmatization=True)

    reduction = (original_tokens - len(processed_tokens)) / original_tokens * 100
    results[name] = {
        'original': original_tokens,
        'processed': len(processed_tokens),
        'reduction': reduction,
        'sample': processed_tokens[:8]
    }

    print(f"\n📄 {name}:")
    print(f"   Original: {original_tokens} tokens")
    print(f"   Processed: {len(processed_tokens)} tokens ({reduction:.1f}% reduction)")
    print(f"   Sample: {processed_tokens[:8]}")

# Summary table
print(f"\n\n📋 Summary Table")
print(f"{'Text Type':<15} {'Original':<10} {'Processed':<10} {'Reduction':<10}")
print("-" * 50)
for name, data in results.items():
    print(f"{name:<15} {data['original']:<10} {data['processed']:<10} {data['reduction']:<10.1f}%")

🔬 Comprehensive Preprocessing Analysis

📄 Simple:
   Original: 14 tokens
   Processed: 7 tokens (50.0% reduction)
   Sample: ['natural', 'language', 'processing', 'fascinating', 'field', 'ai', 'amazing']

📄 Academic:
   Original: 61 tokens
   Processed: 26 tokens (57.4% reduction)
   Sample: ['dr', 'smith', 'research', 'machinelearning', 'algorithm', 'groundbreake', 'publish', 'paper']

📄 Social Media:
   Original: 22 tokens
   Processed: 10 tokens (54.5% reduction)
   Sample: ['omg', 'try', 'new', 'coffee', 'shop', 'good', 'highly', 'recommend']

📄 News:
   Original: 51 tokens
   Processed: 25 tokens (51.0% reduction)
   Sample: ['stock', 'market', 'experience', 'significant', 'volatility', 'today', 'tech', 'stock']

📄 Product Review:
   Original: 47 tokens
   Processed: 18 tokens (61.7% reduction)
   Sample: ['laptop', 'absolutely', 'fantastic', 've', 'use', 'month', 'super', 'fast']


📋 Summary Table
Text Type       Original   Processed  Reduction 
----------------------------------

### 🤔 Final Conceptual Question 13
**Looking at the comprehensive analysis results across all text types:**

1. **Which text type was most affected by preprocessing?** Why do you think this happened?

2. **Which text type was least affected?** What does this tell you about the nature of that text?

3. **If you were building an NLP system to analyze customer reviews for a business, which preprocessing approach would you choose and why?**

4. **What are the main trade-offs you need to consider when choosing preprocessing techniques for any NLP project?**

*Double-click this cell to write your answer:*

**1. Most affected text type:** Product reviews experienced the largest decrease (61.7%) due to their combination of ratings, stop words, and list-style comments, which are aggressively removed during preprocessing.

**2. Least affected text type:** The simple text saw the smallest reduction at 50.0%, indicating it is already concise and free of unnecessary elements such as URLs, emojis, or data that would typically require extensive cleanup.

**3. For customer review analysis:** I would use a standard pipeline to remove stop words and punctuation, apply lemmatization, but retain sentiment-rich adjectives and any rating expressions. This way, I preserve the words that convey opinion while eliminating noise.

**4. Main trade-offs to consider:** We must find a balance between noise reduction and preserving meaningful nuances, speed and linguistic depth, as well as vocabulary size and model complexity. Each decision influences both our accuracy and the efficiency of the system.

---

## 🎯 Lab Summary and Reflection

Congratulations! You've completed a comprehensive exploration of NLP preprocessing techniques.

### 🔑 Key Concepts You've Mastered:

1. **Text Preprocessing Fundamentals** - Understanding why preprocessing is crucial
2. **Tokenization Techniques** - NLTK vs spaCy approaches and their trade-offs
3. **Stop Word Management** - When to remove them and when to keep them
4. **Morphological Processing** - Stemming vs lemmatization for different use cases
5. **Text Cleaning Strategies** - Basic vs advanced cleaning for different text types
6. **Pipeline Design** - Building modular, configurable preprocessing systems

### 🎓 Real-World Applications:
These techniques form the foundation for search engines, chatbots, sentiment analysis, document classification, machine translation, and information extraction systems.

### 💡 Key Insights to Remember:
- **No Universal Solution**: Different NLP tasks require different preprocessing approaches
- **Trade-offs Are Everywhere**: Balance information preservation with noise reduction
- **Context Matters**: The same technique can help or hurt depending on your use case
- **Experimentation Is Key**: Always test and measure impact on your specific task

---

**Excellent work completing Lab 02!** 🎉

For your reflection journal, focus on the insights you gained about when and why to use different techniques, the challenges you encountered, and connections you made to real-world applications.